In [1]:
# The code was removed by DSX for sharing.

In [2]:
if (! require(devtools))
    {
    install.packages('devtools')
    library('devtools')
    }
install_github('IBMDataScience/objectStoreR')
library ('objectStoreR')

Loading required package: devtools
Skipping install of 'objectStoreR' from a github remote, the SHA1 (d6a07c5b) has not changed since last install.
  Use `force = TRUE` to force installation


In [3]:
fileURL <- "https://spark-public.s3.amazonaws.com/dataanalysis/loansData.csv?accessType=DOWNLOAD"
cat ("file URL:" , fileURL, "\n")
download.file(fileURL,destfile="./loansData.csv",mode="w")
loanDataRaw <- read.csv("./loansData.csv",as.is=TRUE)

file URL: https://spark-public.s3.amazonaws.com/dataanalysis/loansData.csv?accessType=DOWNLOAD 


In [4]:
cat("taille du fichier importé: ", dim(loanDataRaw), "\n")
str (loanDataRaw)
write.csv(loanDataRaw,file="loansData.csv")
status <- objectStore.put(credentials_1,"loansData.csv")
status

taille du fichier importé:  2500 14 
'data.frame':	2500 obs. of  14 variables:
 $ Amount.Requested              : int  20000 19200 35000 10000 12000 6000 10000 33500 14675 7000 ...
 $ Amount.Funded.By.Investors    : num  20000 19200 35000 9975 12000 ...
 $ Interest.Rate                 : chr  "8.90%" "12.12%" "21.98%" "9.99%" ...
 $ Loan.Length                   : chr  "36 months" "36 months" "60 months" "36 months" ...
 $ Loan.Purpose                  : chr  "debt_consolidation" "debt_consolidation" "debt_consolidation" "debt_consolidation" ...
 $ Debt.To.Income.Ratio          : chr  "14.90%" "28.36%" "23.81%" "14.30%" ...
 $ State                         : chr  "SC" "TX" "CA" "KS" ...
 $ Home.Ownership                : chr  "MORTGAGE" "MORTGAGE" "MORTGAGE" "MORTGAGE" ...
 $ Monthly.Income                : num  6542 4583 11500 3833 3195 ...
 $ FICO.Range                    : chr  "735-739" "715-719" "690-694" "695-699" ...
 $ Open.CREDIT.Lines             : int  14 12 14 10 11 17 10 1

[1] 201

In [5]:
# nettoyage
colnames (loanDataRaw)[10] <- "FICO.Score"
names(loanDataRaw)
loanDataRaw$Interest.Rate <- gsub("%", "", as.character(loanDataRaw$Interest.Rate))
loanDataRaw$Debt.To.Income.Ratio <- gsub("%", "", as.character(loanDataRaw$Debt.To.Income.Ratio))
str(loanDataRaw)

[1] "Amount.Requested"               "Amount.Funded.By.Investors"    
 [3] "Interest.Rate"                  "Loan.Length"                   
 [5] "Loan.Purpose"                   "Debt.To.Income.Ratio"          
 [7] "State"                          "Home.Ownership"                
 [9] "Monthly.Income"                 "FICO.Score"                    
[11] "Open.CREDIT.Lines"              "Revolving.CREDIT.Balance"      
[13] "Inquiries.in.the.Last.6.Months" "Employment.Length"

'data.frame':	2500 obs. of  14 variables:
 $ Amount.Requested              : int  20000 19200 35000 10000 12000 6000 10000 33500 14675 7000 ...
 $ Amount.Funded.By.Investors    : num  20000 19200 35000 9975 12000 ...
 $ Interest.Rate                 : chr  "8.90" "12.12" "21.98" "9.99" ...
 $ Loan.Length                   : chr  "36 months" "36 months" "60 months" "36 months" ...
 $ Loan.Purpose                  : chr  "debt_consolidation" "debt_consolidation" "debt_consolidation" "debt_consolidation" ...
 $ Debt.To.Income.Ratio          : chr  "14.90" "28.36" "23.81" "14.30" ...
 $ State                         : chr  "SC" "TX" "CA" "KS" ...
 $ Home.Ownership                : chr  "MORTGAGE" "MORTGAGE" "MORTGAGE" "MORTGAGE" ...
 $ Monthly.Income                : num  6542 4583 11500 3833 3195 ...
 $ FICO.Score                    : chr  "735-739" "715-719" "690-694" "695-699" ...
 $ Open.CREDIT.Lines             : int  14 12 14 10 11 17 10 12 9 8 ...
 $ Revolving.CREDIT.Balance      : 

In [6]:
# split du score FICO sur 2 colonnes dans une nouvelle var
ficoSplitScore <- data.frame(do.call(rbind,strsplit(as.vector(loanDataRaw$FICO.Score),split='-')))
# transformation en numeric
names(ficoSplitScore) <- c("FICO.Low","FICO.High")
ficoSplitScore$FICO.Low <- as.numeric(levels(ficoSplitScore$FICO.Low))[ficoSplitScore$FICO.Low]
ficoSplitScore$FICO.High <- as.numeric(levels(ficoSplitScore$FICO.High))[ficoSplitScore$FICO.High]
# str (ficoSplitScore)

In [7]:
# ajout de la nouvelle colonne retravaillée

In [8]:
loanDataRaw <- cbind(loanDataRaw,ficoSplitScore)
# str(loanDataRaw)

In [9]:
# calcul et ajout de la moyenne des 2 colonnes FICO
ficoAvgScore <- data.frame(rowMeans(ficoSplitScore, na.rm=TRUE))
names(ficoAvgScore) <- c("FICO.Avg")
loanDataRaw <- cbind(loanDataRaw,ficoAvgScore)
str(loanDataRaw)

'data.frame':	2500 obs. of  17 variables:
 $ Amount.Requested              : int  20000 19200 35000 10000 12000 6000 10000 33500 14675 7000 ...
 $ Amount.Funded.By.Investors    : num  20000 19200 35000 9975 12000 ...
 $ Interest.Rate                 : chr  "8.90" "12.12" "21.98" "9.99" ...
 $ Loan.Length                   : chr  "36 months" "36 months" "60 months" "36 months" ...
 $ Loan.Purpose                  : chr  "debt_consolidation" "debt_consolidation" "debt_consolidation" "debt_consolidation" ...
 $ Debt.To.Income.Ratio          : chr  "14.90" "28.36" "23.81" "14.30" ...
 $ State                         : chr  "SC" "TX" "CA" "KS" ...
 $ Home.Ownership                : chr  "MORTGAGE" "MORTGAGE" "MORTGAGE" "MORTGAGE" ...
 $ Monthly.Income                : num  6542 4583 11500 3833 3195 ...
 $ FICO.Score                    : chr  "735-739" "715-719" "690-694" "695-699" ...
 $ Open.CREDIT.Lines             : int  14 12 14 10 11 17 10 12 9 8 ...
 $ Revolving.CREDIT.Balance      : 

In [10]:
# ecriture resultat
loanDataFinal <- as.data.frame (loanDataRaw)
write.csv(loanDataFinal, file="loanDataFinal.csv")
status <- objectStore.put(credentials_1,"loanDataFinal.csv")
status

[1] 201